# Decision Tree Lesson

In [ ]:
# General DS Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Decision Tree and Model Evaluation Imports
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

### Question: Can we predict whether someone is a smoker based on the tips data?

In [ ]:
np.random.seed(25)
tips = sns.load_dataset('tips')

In [ ]:
tips.head()

In [ ]:
tips = pd.get_dummies(tips, columns=['sex', 'day', 'time'], drop_first=True)
tips.head()

In [ ]:
X = tips[['total_bill', 'tip', 'size']]
y = tips.smoker

X_train_and_validate, X_test, y_train_and_validate, y_test = train_test_split(X, y, random_state=123, test_size=.3)
X_train, X_validate, y_train, y_validate = train_test_split(X_train_and_validate, y_train_and_validate, random_state=123, test_size=.2)

In [ ]:
X_train.head()

In [ ]:
y_train[:5]

In [ ]:
y_train.value_counts()

## Decision Tree Modeling

#### Step 1: Create Your Recipe

In [ ]:
# Create the decision tree object and specify hyperparams
tree = DecisionTreeClassifier(max_depth = 3)

#### Step 2: Fit your model to your in-sample data (train)

Until a model learns from your data, its just an algorithm. After it has learned and changed its algorithmic structure (i.e. has decided which questions to ask and in what order), then it can be called a model.

In [ ]:
tree.fit(X_train, y_train)

# Rudimentary visualization of model structure
print(export_text(tree, feature_names=X_train.columns.tolist()))

In [ ]:
# Visualize the tree
plt.figure(figsize=(12, 7))
plot_tree(tree, feature_names=X_train.columns, class_names=y_train.unique())
plt.show()

#### Step 3: Use your model to make predictions on the in-sample data

In [ ]:
tree.predict(X_train)

In [ ]:
predictions = tree.predict(X_train)
actual = y_train

#### Step 4: Evaluate your predictions

In [ ]:
confusion_matrix(actual, predictions)

In [ ]:
pd.crosstab(actual, predictions)

In [ ]:
plot_confusion_matrix(tree, X_train, y_train, display_labels=['Non-Smoker', 'Smoker'])

In [ ]:
print(classification_report(actual, predictions))

#### Step 5: Use best n models on out-of-sample data (validate)

In this case, I only have one model

In [ ]:
predictions = tree.predict(X_validate)
actual = y_validate

print(classification_report(actual, predictions))

#### Step 6: Use single best model to make predictions from out-of-sample data (test)

In [ ]:
predictions = tree.predict(X_test)
actual = y_test

print(classification_report(actual, predictions))

#### Bonus: What happens if we change some of the hyperparameters?

Let's try changing max depth to none. This model will ask questions until every leaf gini is as small as absolutely possible:

In [ ]:
# Create the decision tree object and specify hyperparams
tree = DecisionTreeClassifier(max_depth=None) 
tree.fit(X_train, y_train)

print(export_text(tree, feature_names=X_train.columns.tolist()))

In [ ]:
plt.figure(figsize=(16, 9), dpi=300)
plot_tree(tree, feature_names=X_train.columns, class_names=y_train.unique())
plt.show()

#### How accurate is our crazy model on our training data?

In [ ]:
tree.score(X_train, y_train) # 100%!

#### How accurate is our crazy model on out-of-sample data?

In [ ]:
tree.score(X_validate, y_validate) 

In [ ]:
y_validate.value_counts(normalize=True)

We would have had better accuracy just guessing "No" on everything

## What did we miss? What can we do next?

- Making a baseline model and properly evaluating it
- Make more models, try out different combinations of independent variables and hyperparameters
> **Note**: Our X can change, but our y remains constant
- Quantify and compare those models (on validate!) against each other and the baseline

## Pruning
Decision Trees have a high risk of being overfit to our training data. We can use `max_depth` and `min_samples` as a way to reduce overfitting. We could build a process that exhaustively checked every hyperparameter combination or we can use an existing process:
- Look into [`cost_complexity_pruning_path`](https://scikit-learn.org/stable/auto_examples/tree/plot_cost_complexity_pruning.html)

# Exercises

Using the titanic data, in your classification-exercises repository, create a notebook, `model.ipynb` where you will do the following:

1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

5. Run through steps 2-4 using a different max_depth value.

6. Which model performs better on your in-sample data?

7. Which model performs best on your out-of-sample data, the validate set?

8. Work through these same exercises using the Telco dataset.